In [1]:
!pip install bertopic

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# import necessary libraries
from transformers import BertTokenizer, BertModel
import torch
from nltk.cluster import KMeansClusterer
import nltk
import numpy as np
import pandas as pd
import csv
from sklearn.ensemble import IsolationForest
import os
from sklearn.preprocessing import MinMaxScaler

# Input example permission data

In [10]:
import chardet

file_path = "../../Result/P1/topic_clustering_reassigned_result.csv"
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']
permission_file_df = pd.read_csv(file_path, encoding=encoding)

# Replace the specified phrases with an empty string
phrases_to_remove = [
    "Alexa Notifications",
    "Timers",
    "Lists Read Access",
    "Lists Write Access",
    "Reminders",
]

for phrase in phrases_to_remove:
    permission_file_df['Access'] = permission_file_df['Access'].str.replace(phrase, '')

# Remove rows containing only commas, spaces or empty strings
permission_file_df = permission_file_df[~permission_file_df['Access'].str.strip().replace(',', '').replace(' ', '').replace(', ,', '').eq('')]

# Reset the index after removing rows
permission_file_df.reset_index(drop=True, inplace=True)
permission_file_df['Access'] = permission_file_df['Access'].str.strip(',')


0            , Amazon Pay
1            , Amazon Pay
2            , Amazon Pay
3            , Amazon Pay
4            , Amazon Pay
              ...        
1493        Mobile Number
1494        Mobile Number
1495        Mobile Number
1496        Mobile Number
1497     Skill Resumption
Name: Access, Length: 1498, dtype: object


# Classfication

In [11]:
import torch
from transformers import BertTokenizer, BertModel

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)
model.eval()  # Set the model to evaluation mode


def bert_encoder(sentence_list):
    inputs = tokenizer(sentence_list, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    CLS_token_embeddings = outputs[0][:, 0, :].cpu().data.numpy()
    return CLS_token_embeddings


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Bert embedding for entire line:**

In [18]:
def app(topic_num):
  df = permission_file_df[permission_file_df.topic == topic_num][["Index", "Skill Path", "Access", "target_text"]]
  iso_forest_file = permission_file_df[permission_file_df.topic == topic_num][["Index", "Skill Path", "Access","target_text"]]

  lines = df['Access'].tolist()
  emb = bert_encoder(lines)

  #use IF to detect
  dataset = iso_forest_file
  dataset.columns = ['Index','Skill Path','Access',"target_text"]

  #set variable
  rs = np.random.RandomState(64)
  lendata = dataset.shape[0]
  test_data = []
  nmlz_a = -1; nmlz_b = 1

  #read dataset voltage
  x = np.array(emb)
  ifm = IsolationForest(n_estimators=100, verbose=2, n_jobs=2,
                        max_samples=lendata, random_state=rs, max_features=1)

  Iso_train_dt = np.column_stack(x)
  ifm.fit(x)
  scores_pred = ifm.decision_function(x)
  print(type(scores_pred))

  print(topic_num)
  iso_threshold = find_best_threshold_kmeans(x, scores_pred)

  iso_file_path = f"../../Result/P2/consistency_checking_result/Iso_demo[{topic_num}_{iso_threshold}].csv"

  for i in scores_pred:

      if i <= iso_threshold:
          test_data.append(1)
      else:
          test_data.append(0)

  index = list(range(0, len(test_data)))

  ano_lable = np.column_stack((dataset['Index'],dataset['Skill Path'],dataset['Access'], dataset['target_text'],scores_pred, test_data))

  true_matrix = []
  count = 0
  for rows in ano_lable:
      true_matrix.append(rows)
      count = count + 1
  df = pd.DataFrame(data=true_matrix, columns=['Index','Skill Path','Access', "target_text",'IsoFst_Score', 'label'])

  plotlist = df.to_csv(iso_file_path)


**Auto determine the best threshold value**

In [19]:
from sklearn.cluster import KMeans

def find_best_threshold_kmeans(data, anomaly_scores, n_clusters=3):
    n_clusters = min(n_clusters, len(anomaly_scores))

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(anomaly_scores.reshape(-1, 1))

    avg_scores = [np.mean(anomaly_scores[cluster_labels == i]) for i in range(n_clusters)]

    outlier_cluster = np.argmin(avg_scores)

    outlier_samples = anomaly_scores[cluster_labels == outlier_cluster]
    if len(outlier_samples) == 0:
        best_threshold = -0.06
    else:
        best_threshold = np.max(outlier_samples)

    return best_threshold





# Outlier Detection



In [20]:
distinct_values = permission_file_df['topic'].value_counts()

# Filter out values with count <= 2
filtered_values = distinct_values[distinct_values > 2]
filtered_list = filtered_values.index.tolist()

for i in filtered_list:
  app(i)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Building estimator 1 of 50 for this parallel run (total 100)...
Building estimator 1 of 50 for this parallel run (total 100)...
Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this parallel run (total 100)...
Building estimator 8 of 50 for this parallel run (total 100)...
Building estimator 8 of 50 for this para

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Building estimator 1 of 50 for this parallel run (total 100)...
Building estimator 1 of 50 for this parallel run (total 100)...
Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this parallel run (total 100)...
Building estimator 8 of 50 for this parallel run (total 100)...
Building estimator 8 of 50 for this para

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
/home/ximena/.local/lib/python

Building estimator 1 of 50 for this parallel run (total 100)...Building estimator 1 of 50 for this parallel run (total 100)...

Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 2 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 3 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 4 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 5 of 50 for this parallel run (total 100)...
Building estimator 6 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this parallel run (total 100)...
Building estimator 8 of 50 for this parallel run (total 100)...
Building estimator 9 of 50 for this parallel run (total 100)...
Building estimator 7 of 50 for this para

[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
/home/ximena/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


KeyboardInterrupt: 

Building estimator 50 of 50 for this parallel run (total 100)...
